In [ ]:
import math

In [ ]:
config = {
    'f1': 50, # основная частота питания 50 Гц
    'f_sampling': 10000, # частота дискретизации 10 кГц
    's': 0.028, # скольжение двигателя 2.8% (справочное), 2.67% (расчётное)
    'R_s': 34, # число пазов (стержней) ротора
    'p': 4, # число пар полюсов (заменил на f_r/f1)
    'f_r': 12.333, # частота вращения ротора (вала) 1460 RPM в Гц
    'n': 8, # число шариков в подшипнике
    'beta': 0, # угол контакта
    'D_pit': 15.0810, # диаметр окружности центра шариков (мм)
    'D_ball': 65 # диаметр шарика (мм)
    }

In [ ]:
def get_type1_freqs(engine_config, n_range=range(1,4)):
    # частоты для дефекта клетки ротора. n_range - диапазон гармоник
    s = engine_config['s']
    f1 = engine_config['f1']
    freqs = []
    for n in n_range:
        assert n > 0
        assert isinstance(n, int)
        freqs.append((1+2*n*s)*f1)
        freqs.append((1-2*n*s)*f1)
    return freqs


def get_type2_freqs(engine_config, n_range=range(1,4)):
    # частоты для эксцентриситета воздушного зазора. n_range - диапазон гармоник
    R_s = engine_config['R_s']
    p = engine_config['p']
    s = engine_config['s']
    f1 = engine_config['f1']
    freqs = []
    for n in n_range:
        assert n > 0
        assert isinstance(n, int)
        freqs.append(f1*((R_s*(1-s)/p)+n+((1-s)/p)))
        freqs.append(f1*((R_s*(1-s)/p)+n-((1-s)/p)))
        freqs.append(f1*((R_s*(1-s)/p)-n+((1-s)/p)))
        freqs.append(f1*((R_s*(1-s)/p)-n-((1-s)/p)))
    return freqs

def get_type3_freqs(engine_config, n_range=range(1,4), k_range=range(1,4)):
    # функция расчёта гармоник для межвитковых замыканий. n_range и k_range - диапазон для гармоник
    f1 = engine_config['f1']
    p = engine_config['p']
    s = engine_config['s']
    freqs = []
    for n in n_range:
        assert n > 0
        assert isinstance(n, int)
        for k in k_range:
            assert k > 0
            assert isinstance(k, int)
            freqs.append(f1*((n*(1-s)/p)+k))
            freqs.append(f1*((n*(1-s)/p)-k))
    return freqs

def get_type4_1_freqs(engine_config):
    # функция расчёта гармоник для дефектов подшипника (тело качения)
    D_pit = engine_config['D_pit']
    D_ball = engine_config['D_ball']
    f_r = engine_config['beta']
    beta = engine_config['beta']
    return [(D_pit/D_ball)*f_r*(1-(D_ball/(D_pit*math.cos(beta)))**2)]

def get_type4_2_freqs(engine_config):
    # функция расчёта гармоник для дефектов подшипника (внешняя дорожка)
    n = engine_config['n']
    D_pit = engine_config['D_pit']
    D_ball = engine_config['D_ball']
    f_r = engine_config['beta']
    beta = engine_config['beta']
    return [(n/2)*f_r*(1-(D_ball/(D_pit*math.cos(beta))))]

def get_type4_3_freqs(engine_config):
    # функция расчёта гармоник для дефектов подшипника (внутренняя дорожка)
    n = engine_config['n']
    D_pit = engine_config['D_pit']
    D_ball = engine_config['D_ball']
    f_r = engine_config['beta']
    beta = engine_config['beta']
    return [(n/2)*f_r*(1+(D_ball/(D_pit*math.cos(beta))))]


def get_type5_freqs(engine_config, n_range=range(1,4)):
    # функция расчёта гармоник для других механических поломок. n_range - диапазон для гармоник
    f_r = engine_config['f_r']
    f1 = engine_config['f1']
    freqs = []
    for n in n_range:
        assert n > 0
        assert isinstance(n, int)
        freqs.append(f1 + n*f_r)
    return freqs



In [ ]:
get_type1_freqs = get_type1_freqs(config)
get_type2_freqs = get_type2_freqs(config)
get_type3_freqs = get_type3_freqs(config)
get_type4_1_freqs = get_type4_1_freqs(config)
get_type4_2_freqs = get_type4_2_freqs(config)
get_type4_3_freqs = get_type4_3_freqs(config)
get_type5_freqs = get_type5_freqs(config)

In [ ]:
ANOMALY_FREQS = {'rotor cell defect': get_type1_freqs, 'air gap eccentricity': get_type2_freqs,
                 'inter-turn short circuits': get_type3_freqs, 'bearing defect (rolling element)': get_type4_1_freqs,
                 'bearing defect (outer race)': get_type4_2_freqs, 'bearing defect (inner race)': get_type4_3_freqs,
                 'other mechanical defects': get_type5_freqs}


In [ ]:
ANOMALY_FREQS['rotor cell defect']

[52.800000000000004, 47.199999999999996, 55.60000000000001, 44.4, 58.4, 41.6]